# Import stuff

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
import nltk
nltk.download('punkt')
from itertools import groupby
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# MongoDB Load data

In [11]:
!pip install pymongo
from pymongo import MongoClient
client = MongoClient('192.168.80.61')
#client = MongoClient('103.199.7.250')

In [12]:
client.list_database_names()

['admin',
 'admin_manager',
 'config',
 'cv-matching',
 'local',
 'qnujc-cms-db',
 'resthub',
 'strapi']

In [16]:
db=client['cv-matching']

In [18]:
db.list_collection_names()
#db.collection.explain() 

cvsCol = db.cvs_raw
cvsCol

Collection(Database(MongoClient(host=['192.168.80.61:27017'], document_class=dict, tz_aware=False, connect=True), 'cv-matching'), 'cvs_raw')

In [19]:
cursor = cvsCol.find().limit(1)
for doc in cursor:
    print (doc)

{'_id': ObjectId('5adeafd0c4f9737ed57ef416'), 'cid': 2091763, 'name': 'Phạm Uyên Tri', 'general_info': {'expected_job_level': 2, 'expected_position': 'PHP Developer', 'experience_level': 2, 'highest_education': 0, 'job_domain': None, 'job_industries_functions': ['IT - Software'], 'job_locations': ['Ho Chi Minh'], 'languages': [{'language': 'English', 'level': 2}], 'minimum_expected_salary': 1000, 'years_of_experience': 4, 'most_recent_job': 'PHP Developer', 'latest_company': 'Techbase Viet Nam'}, 'contact_info': {'phone': None, 'email': 'phamuyentri@gmail.com', 'gender': 1, 'birthday': datetime.datetime(1987, 11, 7, 0, 0)}, 'educations': [], 'skills': ['Selenium Webdriver', 'Phpunit', 'Angula JS Developer', 'Codeception', 'Fuel Framework', 'PHP', 'mysql', 'HTML', 'CSS', 'Javascript', 'Bootstrap...', 'Phonegap', 'Ionic Framework', 'Service Stack', 'Phalcon Framework', 'C#', '.NET', 'Svn', 'Git'], 'last_update': datetime.datetime(2018, 1, 1, 0, 0), 'link': 'https://employer.vietnamworks.

In [21]:
pipeline = [
        {"$group": {"Position":"$general_info.expected_position",
             "Count":{"$sum":1},
        }},
        {"$sort":{"Count":-1,"Position":2}}
]
aggResult = cvsCol.aggregate(pipeline) # returns a cursor
df1 = pd.DataFrame(list(aggResult)) # use list to turn the cursor to an array of documents
df1 = df1.set_index("_id")
df1.head()

OperationFailure: The field 'Position' must be an accumulator object

In [ ]:
cs = cvsCol.find()

In [ ]:
csarr = []
i=0
for c in cvsCol.find():
    csarr.append(c)
    i = i+1
    if i%1000==0 :
        print(".", end ="" )
    

In [ ]:
len(csarr)

In [ ]:

import pickle
file_path = 'c:/tmp/cvs.pkl'
pickle.dump(csarr, open(file_path, 'wb'))

# Load "saved" pickle file

In [ ]:
pickle_in = open(file_path,"rb")
example_dict = pickle.load(pickle_in)

In [ ]:
len(example_dict)

# Get the Job "Expected Position"

In [ ]:
type(example_dict[0])

In [ ]:
example_dict[0].keys()

In [1]:
expPos = []
l0 = len(example_dict)
for cv in example_dict:
    expPos.append(cv['general_info']['expected_position'])
l1 = len(expPos)
expPos = {x for x in expPos if x is not None}
l2 = len(expPos)
expPos = sorted(expPos)
l3 = len(expPos)
l0,l1,l2,l3

NameError: name 'example_dict' is not defined

# Extract the job phrases and words

In [380]:

#res = [(key, len(list(group))) for key, group in groupby(expPos)]# or {key: len(list(group)) for key, group in groupby(expPos)}
phrases = expPos # [key for key, group in groupby(expPos)]# or {key: len(list(group)) for key, group in groupby(expPos)}

len(phrases)

7468

In [390]:
import re
delimiters = ",","&","!","|",":",";",'\u200b', "...", "[", "]", "(", ")", ". ", " + ", " - ", ". ", "{", "}", "$", "?", "*", "~", '’', "/", "\\", "(c)"

def split(string, maxsplit=0):
    
    regexPattern = '|'.join(map(re.escape, delimiters))
    return re.split(regexPattern, string, maxsplit)

delimiters = ",","&","!","|",":",";",'\u200b',"'",'"', "...", "[", "]", "(", ")", ". ", " + ", " - ", ". ", "{", "}", "$", "?", "*", "~", '’', "/", "\\", "(c)"

def split_sentence(string, maxsplit=0):
    import re
    regexPattern = '|'.join(map(re.escape, delimiters))
    return re.split(regexPattern, string, maxsplit)


#import nltk
#nltk.download('punkt')
phrasesTok = []
phrasesWords = []
phrasesText = []
for p in phrases:
    #words = nltk.word_tokenize(p.strip().lower())
    phraseparts = split(p.strip().lower())
    words = []
    if(len(phraseparts)>0):
        for w in phraseparts:
            w = w.strip()
            if(len(w)>0):
                phrasesText.append(w)
                ws = w.split(' ')
                
                if(len(ws)>1):
                    for wsw in ws:
                        if(len(wsw)>0):
                            phrasesWords.append(wsw)
                else:
                    if(len(w)>0):
                        phrasesWords.append(w)
        phrasesTok.append(words)
#phrasesTok

#phrasesText

In [391]:
phrasesWords = sorted(phrasesWords)
phraseWordCount = []
for key, igroup in itertools.groupby(phrasesWords, lambda x: x ):
    if(len(key)>0): # skip empty values
        phraseWordCount.append([key, len(list(igroup))])
    #print (key, len(list(igroup)))
    

phraseWordCount = sorted(phraseWordCount, key=lambda x: x[1], reverse=True)
nonAlpha = [w for w in phraseWordCount if len(w[0])==1 and not ((w[0][0]).isalpha())]
#nonAlpha

In [392]:
phrasesText = sorted(phrasesText)
phraseTextCount = []
for key, igroup in itertools.groupby(phrasesText, lambda x: x ):
    if(len(key)>0): # skip empty values
        phraseTextCount.append([key, len(list(igroup))])
    #print (key, len(list(igroup)))
    

phraseTextCount = sorted(phraseTextCount, key=lambda x: x[1], reverse=True)
nonAlphaPT = [w for w in phraseWordCount if len(w[0])==1 and not ((w[0][0]).isalpha())]

In [402]:
#phraseTextCount

In [393]:
#nonAlpha = [w for w in phraseWordCount if len(w[0])<1]
#nonAlpha
#phrasesWords
#phraseWordCount

In [422]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

asciiWords = [w for w in phraseWordCount if is_ascii(w[0])]
asciiPhraseTexts = [w for w in phraseTextCount if is_ascii(w[0])]

nonAsciiWords = [w for w in phraseWordCount if not is_ascii(w[0])]
nonAsciiPhraseTexts = [w for w in phraseTextCount if not is_ascii(w[0])]
#nonAsciiWords
len(asciiWords), len(nonAsciiWords), len(phraseTextCount),len(asciiPhraseTexts), len(nonAsciiPhraseTexts)
#nonAsciiPhraseTexts
asciiWords

[['developer', 1141],
 ['manager', 940],
 ['it', 809],
 ['senior', 680],
 ['engineer', 587],
 ['software', 516],
 ['leader', 455],
 ['project', 405],
 ['java', 315],
 ['tester', 306],
 ['analyst', 298],
 ['business', 287],
 ['team', 286],
 ['web', 285],
 ['php', 235],
 ['.net', 219],
 ['technical', 213],
 ['consultant', 205],
 ['lead', 203],
 ['qa', 187],
 ['system', 176],
 ['and', 170],
 ['erp', 167],
 ['qc', 167],
 ['android', 158],
 ['designer', 153],
 ['support', 140],
 ['marketing', 137],
 ['development', 124],
 ['director', 118],
 ['product', 112],
 ['ios', 109],
 ['test', 107],
 ['data', 106],
 ['specialist', 103],
 ['administrator', 101],
 ['fresher', 100],
 ['mobile', 99],
 ['of', 99],
 ['or', 98],
 ['architect', 97],
 ['sap', 96],
 ['c++', 94],
 ['application', 93],
 ['doanh', 93],
 ['khai', 93],
 ['kinh', 91],
 ['sales', 91],
 ['c#', 90],
 ['executive', 88],
 ['ba', 87],
 ['database', 85],
 ['sinh', 84],
 ['game', 80],
 ['tin', 80],
 ['junior', 79],
 ['dev', 77],
 ['assistan

In [421]:
job="rf"
jobMatches = []
for w in job.strip().lower().split():
    jobMatches.append( [m for m in asciiPhraseTexts if(w in m[0].split())])

jobMatches

[[['rf analog design engineer', 1],
  ['rf optimization engineering.', 1],
  ['sw rf engineer', 1]]]

In [219]:
results = [t for t in multiple if t[1]>1]

In [211]:
results

[]